# Lesson 2 – Multiple Inputs Pattern: Daily Expense Summary Agent

This notebook is part of the **LangGraph Agentic AI – Intro Course**.

In this lesson, you will build a small LangGraph app that takes **multiple inputs** (a list of expenses, currency, and budget) and returns a human-friendly daily summary.


## 1. Objectives & Prerequisites

**Objectives**

By the end of this lesson, you can:

- Design a state with **multiple input fields**.
- Implement a node that computes derived values (total spent, number of transactions, remaining budget).
- Wrap the node in a LangGraph `StateGraph` and run it.
- Extend the state and node to support richer behavior (currencies, categories, alerts).

**Prerequisites**

- Lesson 1 completed (single-node pattern).
- Comfortable with Python lists and simple arithmetic.


## 2. Environment Setup

Make sure you have installed the course dependencies (from the repo root):

```bash
python -m venv .venv
source .venv/bin/activate   # on Windows: .venv\Scripts\activate
pip install -r env/requirements.txt
```

Then start Jupyter and select the `langgraph-intro` (or equivalent) Python kernel.


## 3. Concept Warm-up

In Lesson 1, our agent state stored just one main input (`original_subject`) and one output (`improved_subject`).

In many real scenarios, an agent needs to work with **multiple inputs** at once. For example, a daily expense summary might need:

- A list of individual expense amounts.
- The currency used.
- A daily budget.

From this, the agent can compute:

- Total spent today.
- Number of transactions.
- Whether you're under or over your budget.
- A short summary sentence.


### Quick scratch cell (optional)

Use this cell for quick experiments while you follow along.


In [ ]:
# Scratch space for experimentation

example_expenses = [12.5, 5.0, 20.0]
sum(example_expenses)

## 4. Define the State

We'll represent the state for this lesson as a `TypedDict` with the following fields:

- **Inputs**:
  - `expenses: List[float]` – individual amounts.
  - `currency: str` – e.g. `"EUR"`, `"USD"`.
  - `daily_budget: float` – the user's target spend for the day.

- **Derived / outputs**:
  - `total_spent: float`
  - `num_transactions: int`
  - `summary: str`


In [ ]:
from typing import TypedDict, List

class ExpenseState(TypedDict):
    expenses: List[float]
    currency: str
    daily_budget: float
    total_spent: float
    num_transactions: int
    summary: str

# Example of an initial state
initial_state: ExpenseState = {
    "expenses": [12.5, 5.0, 20.0],
    "currency": "EUR",
    "daily_budget": 50.0,
    "total_spent": 0.0,
    "num_transactions": 0,
    "summary": ""
}

initial_state

## 5. Implement the Summarizer Node

Our node should:

1. Read `expenses`, `currency`, and `daily_budget` from the state.
2. Compute:
   - `total_spent` = sum of expenses.
   - `num_transactions` = length of the list.
   - A simple status message depending on how close we are to `daily_budget`.
3. Write `total_spent`, `num_transactions`, and `summary` back to the state.
4. Return the updated state.


In [ ]:
def summarize_expenses(state: ExpenseState) -> ExpenseState:
    """Summarize the user's daily expenses.

    - Computes total spent and number of transactions.
    - Compares against daily budget.
    - Writes a human-readable summary string into the state.
    """
    expenses = state["expenses"]
    total = sum(expenses)
    num = len(expenses)
    budget = state["daily_budget"]
    currency = state["currency"]

    remaining = budget - total
    if budget > 0:
        if remaining > 0:
            status = f"You are {remaining:.2f} {currency} under budget."
        elif remaining == 0:
            status = "You hit your budget exactly today."
        else:
            status = f"You are {-remaining:.2f} {currency} over budget."
    else:
        status = "No budget set."

    summary = (
        f"You spent {total:.2f} {currency} today across {num} transactions. {status}"
    )

    state["total_spent"] = total
    state["num_transactions"] = num
    state["summary"] = summary
    return state


# Quick local test without LangGraph
test_state: ExpenseState = {
    "expenses": [12.5, 5.0, 20.0],
    "currency": "EUR",
    "daily_budget": 50.0,
    "total_spent": 0.0,
    "num_transactions": 0,
    "summary": "",
}
summarize_expenses(test_state)

## 6. Build and Run the LangGraph App

Now we'll wrap the `summarize_expenses` node in a LangGraph `StateGraph`.

Steps:

1. Create `StateGraph(ExpenseState)`.
2. Add the `summarize_expenses` node.
3. Set the node as both entry and finish point.
4. Compile to an app and invoke it with an initial state.


In [ ]:
from langgraph.graph import StateGraph

# 1. Create the graph
graph = StateGraph(ExpenseState)

# 2. Add the node
graph.add_node("summarize_expenses", summarize_expenses)

# 3. Set entry and finish points
graph.set_entry_point("summarize_expenses")
graph.set_finish_point("summarize_expenses")  # single-node graph

# 4. Compile the graph
app = graph.compile()

# 5. Invoke with an initial state
result = app.invoke({
    "expenses": [12.5, 5.0, 20.0],
    "currency": "EUR",
    "daily_budget": 50.0,
    "total_spent": 0.0,
    "num_transactions": 0,
    "summary": "",
})
result

Try changing the inputs and rerunning the cell above, for example:

- `expenses = []` (no spend)
- `daily_budget = 0.0` (no set budget)
- Very high spending with a low budget.


## 7. Exercises

Use the cells below to implement the exercises described in the course README.

---

### Exercise 1 – Zero-expense Day

**Goal:**

Handle the case where `expenses` is an empty list:

- `total_spent = 0`
- `num_transactions = 0`
- `summary = "No expenses recorded today."`

Update `summarize_expenses` to include this special case.


In [ ]:
# TODO: Update summarize_expenses to handle zero-expense days.

# Hint:
# if not expenses:
#     ...


### Exercise 2 – Multiple Currencies

**Goal:**

Add support for a `target_currency` and a (fake) conversion rate:

- Extend `ExpenseState` with a `target_currency: str` and `converted_total: float`.
- Write a simple conversion function (e.g. multiply by 1.1).
- Extend `summary` to mention both original and converted totals, like:

> "You spent 37.50 EUR today (~41.25 USD) across 5 transactions. ..."


In [ ]:
# TODO: Extend ExpenseState and summarize_expenses to support target_currency.

# Example idea for conversion:
# def convert(amount: float, rate: float) -> float:
#     return amount * rate


### Stretch Exercise 1 – Category Breakdown

**Goal:**

Change the state structure to support categories for each expense.

For example:

```python
class Expense(TypedDict):
    amount: float
    category: str  # "food", "transport", "entertainment", ...

class ExpenseState(TypedDict):
    expenses: List[Expense]
    ...
```

Then:

- Compute total per category.
- Mention the **top category** in the summary, e.g.:

> "Your top spending category was food (20.00 EUR)."


In [ ]:
# TODO: Introduce per-expense categories and compute per-category totals.

# Hint:
# from collections import defaultdict
# totals_by_category = defaultdict(float)


### Stretch Exercise 2 – Alert Flag

**Goal:**

Add an `alert: bool` field to the state.

- Set `alert = True` if the user is more than 20% over their daily budget.
- Leave it `False` otherwise.

You can also optionally append a warning to the `summary` string when `alert` is `True`.


In [ ]:
# TODO: Add an 'alert' flag when the user is significantly over budget.

# Example condition:
# if budget > 0 and total > 1.2 * budget:
#     alert = True


---

You’ve now completed the scaffold for **Lesson 2 – Multiple Inputs Pattern**.

Next steps:

- Commit this notebook and your solutions to your Git repo.
- Continue to Lesson 3, where you will build a **sequential multi-node pipeline** for bug report triage.
